In [810]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.naive_bayes import GaussianNB
import math
import copy

In [811]:
df=pd.read_csv("LoanDataset/data.csv")
df.columns = ['id','age','exp','income','zip','family','spend','education','house','output','security','certi','net','creditcard']

In [812]:
X = df.drop(['output'],axis=1)
Y = df['output']

In [813]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)

In [814]:
df = pd.concat([X_train,Y_train],axis=1)

In [815]:
distinct = {}
def seperateClass():
    for index,row in df.iterrows():

        if row['output'] not in distinct:
            distinct[row['output']]=[]
        distinct[row['output']].append(list(row))
        

In [816]:
def mean(attrColumn):
    return float(sum(attrColumn))/len(attrColumn)

In [817]:
def standard_deviation(attrColumn):
    avg = mean(attrColumn)
    ans = 0
    for i in attrColumn:
        ans=ans+pow(i-avg,2)
    ans = float(ans)/(len(attrColumn)-1)
    return np.sqrt(ans)

In [818]:
def find_mean_sd_for_attr(df):

    summaries = {}
    df = pd.DataFrame(df)
    df.columns = ['id','age','exp','income','zip','family','spend','education','house','security','certi','net','creditcard','output']
    for i in df.columns:
        summaries[i]=(mean(df[i]),standard_deviation(df[i]))
    del summaries['output']
    return summaries

In [819]:
seperateClass()
# print distinct
summary = {}
for label,data in distinct.iteritems():
    summary[label] = find_mean_sd_for_attr(data)
    
# print summary

In [820]:
def calcProbability(x,mean,sd):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(sd,2))))
    return (1 / (math.sqrt(2*math.pi) * sd)) * exponent

In [821]:
def probability_by_class(inputdata,summary):

    probabilities = {}
    for label, labeldata in summary.iteritems():
#         print labeldata
        probabilities[label] = 1
        for key,value in labeldata.iteritems():
#             print value
            mean,sd = value
            inputattrdata = inputdata[key]
            probabilities[label] *= calcProbability(inputattrdata, mean, sd)
    return probabilities

In [822]:
def prediction(inputdata,summary):
    probabilities = probability_by_class(inputdata,summary)
    anslabel = ""
    maxprob = 0
    for label, probability in probabilities.iteritems():
        if anslabel=="" or probability >= maxprob:
            maxprob = probability
            anslabel = label
#     print anslabel
    return anslabel

In [823]:
res = []

for index,row in X_test.iterrows():
    res.append(prediction(row,summary))

print confusion_matrix(Y_test,res)
print classification_report(Y_test,res)
print accuracy_score(Y_test, res)*100

[[728  81]
 [ 20  71]]
              precision    recall  f1-score   support

           0       0.97      0.90      0.94       809
           1       0.47      0.78      0.58        91

   micro avg       0.89      0.89      0.89       900
   macro avg       0.72      0.84      0.76       900
weighted avg       0.92      0.89      0.90       900

88.77777777777777


In [824]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
y_pred = gnb.predict(X_test)
print confusion_matrix(Y_test,y_pred)
print classification_report(Y_test,y_pred)
print accuracy_score(Y_test, y_pred)*100

[[752  57]
 [ 39  52]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       809
           1       0.48      0.57      0.52        91

   micro avg       0.89      0.89      0.89       900
   macro avg       0.71      0.75      0.73       900
weighted avg       0.90      0.89      0.90       900

89.33333333333333
